In [95]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision

In [159]:
raw_df = pd.read_csv('../data/car data.txt')
raw_df

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0
...,...,...,...,...,...,...,...,...,...
296,city,2016,9.50,11.60,33988,Diesel,Dealer,Manual,0
297,brio,2015,4.00,5.90,60000,Petrol,Dealer,Manual,0
298,city,2009,3.35,11.00,87934,Petrol,Dealer,Manual,0
299,city,2017,11.50,12.50,9000,Diesel,Dealer,Manual,0


In [160]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

df = raw_df.drop(['Car_Name'], axis=1, )
# Encoding
df['Fuel_Type'] = df['Fuel_Type'].astype('category').cat.codes
df['Seller_Type'] = df['Seller_Type'].astype('category').cat.codes
df['Transmission'] = df['Transmission'].astype('category').cat.codes


In [161]:
# Train Test Split
X, y = df.drop(['Selling_Price'], axis=1), df.Selling_Price

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# df

In [174]:
# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [243]:
import torch
from torch import nn
import torch.nn.functional as F


class CarNN(nn.Module):
    def __init__(self, input_size, output_size, bias=True):
        super(CarNN, self).__init__()
        self.bias = bias
        self.w = nn.Parameter(torch.empty(output_size, input_size))
        if bias:
            self.b = nn.Parameter(torch.empty(output_size))
        self.reset_params()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_size, 300)
        self.r1 = nn.ReLU()
        self.drop1 = nn.Dropout(p=0.2)
        self.fc2 = nn.Linear(300, 100)
        self.r2 = nn.SiLU()
        self.fc3 = nn.Linear(100, 50)
        self.r3 = nn.SELU()
        self.drop2 = nn.Dropout(p=0.2)
        self.fc4 = nn.Linear(50, output_size)
        self.end = nn.Softmax()
    def reset_params(self):
        nn.init.kaiming_normal_(self.w, nonlinearity="relu")
        if self.bias:
            nn.init.constant_(self.b, 0)

    def forward(self, x):
        # x = self.flatten(x)
        x = self.fc1(x)
        x = self.r1(x)
        x = self.drop1(x)
        x = self.fc2(x)
        x = self.r2(x)
        x = self.fc3(x)
        x = self.r3(x)
        x = self.drop2(x)
        x = self.fc4(x)
        x = self.end(x)
        return x
        # return F.linear(x, self.w, self.b)


In [244]:
# Define the hyperparameters
input_size = 7
output_size = 1
learning_rate = 0.001
num_epochs = 1000
label_size = 1
# Create the neural network object
model = CarNN(input_size, output_size)


In [245]:
dict(model.named_parameters())["w"].shape


torch.Size([1, 7])

In [246]:
from torch import optim

criterion = nn.L1Loss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)


In [247]:
X_train_ = torch.tensor(X_train, dtype=torch.float32)
y_train_ = torch.tensor(y_train, dtype=torch.float32)
X_test_ = torch.tensor(X_test, dtype=torch.float32)
# y_test_ = torch.tensor(y_test, dtype=torch.float32)


In [248]:
print(X_train_.size())
print(y_train_.size())
print(X_test_.size())


torch.Size([240, 7])
torch.Size([240])
torch.Size([61, 7])


In [249]:
X_train_

tensor([[-1.9699, -0.7536, -0.2756,  ...,  1.3382,  0.3780,  3.5282],
        [ 1.1564, -0.7313, -0.8143,  ...,  1.3382,  0.3780, -0.1857],
        [-1.9699, -0.7495,  0.2991,  ...,  1.3382,  0.3780, -0.1857],
        ...,
        [ 0.1143, -0.4527, -0.5030,  ...,  1.3382,  0.3780,  3.5282],
        [-0.9278,  0.2773,  0.7622,  ..., -0.7473,  0.3780, -0.1857],
        [ 1.1564, -0.6388, -0.8023,  ...,  1.3382,  0.3780, -0.1857]])

In [252]:

for epoch in range(num_epochs):
    outputs = model(X_train_)
    # print(outputs)
    
    loss = criterion(outputs, y_train_)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch: {epoch + 1}/{num_epochs}, Loss: {loss.item()}')


/home/rahmani/work/envs/ML/lib/python3.11/site-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/home/rahmani/work/envs/ML/lib/python3.11/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([240])) that is different to the input size (torch.Size([240, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


Epoch: 100/1000, Loss: 3.9365413188934326
Epoch: 200/1000, Loss: 3.9365413188934326
Epoch: 300/1000, Loss: 3.9365413188934326
Epoch: 400/1000, Loss: 3.9365413188934326
Epoch: 500/1000, Loss: 3.9365413188934326
Epoch: 600/1000, Loss: 3.9365413188934326
Epoch: 700/1000, Loss: 3.9365413188934326
Epoch: 800/1000, Loss: 3.9365413188934326
Epoch: 900/1000, Loss: 3.9365413188934326
Epoch: 1000/1000, Loss: 3.9365413188934326


In [250]:

model.eval()
with torch.no_grad():
    test_outputs = model(X_test_)
    _, predicted = torch.max(test_outputs.data, 1)
    print("Predictions:", predicted)